In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

import ast

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
import warnings
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.model_selection import train_test_split 

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
charging_set = pd.read_csv("data/charging_sessions.csv")

In [3]:
# get overview of data
print("columns:")
print(charging_set.columns)
print("-------------------------------------------------------------------------")
print("info():")
print(charging_set.info())
print("-------------------------------------------------------------------------")
print("describe():")
print(charging_set.describe())

columns:
Index(['Unnamed: 0', 'id', 'connectionTime', 'disconnectTime',
       'doneChargingTime', 'kWhDelivered', 'sessionID', 'siteID', 'spaceID',
       'stationID', 'timezone', 'userID', 'userInputs'],
      dtype='object')
-------------------------------------------------------------------------
info():
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66450 entries, 0 to 66449
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        66450 non-null  int64  
 1   id                66450 non-null  object 
 2   connectionTime    66450 non-null  object 
 3   disconnectTime    66450 non-null  object 
 4   doneChargingTime  62362 non-null  object 
 5   kWhDelivered      66450 non-null  float64
 6   sessionID         66450 non-null  object 
 7   siteID            66450 non-null  int64  
 8   spaceID           66450 non-null  object 
 9   stationID         66450 non-null  object 
 10  timezone    

In [4]:
charging_set.head(10)

,Unnamed: 0,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,sessionID,siteID,spaceID,stationID,timezone,userID,userInputs
0,0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1_1_179_810_2020-01-02 13:08:53.870034,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1_1_193_825_2020-01-02 13:36:49.599853,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1_1_193_829_2020-01-02 13:56:35.214993,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
3,3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1_1_193_820_2020-01-02 13:59:58.309319,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."
4,4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1_1_193_819_2020-01-02 14:00:00.779967,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,"[{'WhPerMile': 400, 'kWhRequested': 16.0, 'mil..."
5,5,5e23b149f9af8b5fe4b973d4,2020-01-02 14:00:13+00:00,2020-01-02 23:00:41+00:00,2020-01-02 20:13:21+00:00,13.370,1_1_194_821_2020-01-02 14:00:05.241573,1,AG-1F13,1-1-194-821,America/Los_Angeles,3519.0,"[{'WhPerMile': 600, 'kWhRequested': 24.0, 'mil..."
6,6,5e23b149f9af8b5fe4b973d5,2020-01-02 14:09:14+00:00,2020-01-03 01:57:58+00:00,2020-01-03 00:59:21+00:00,43.477,1_1_178_817_2020-01-02 14:09:14.451827,1,AG-1F09,1-1-178-817,America/Los_Angeles,933.0,"[{'WhPerMile': 385, 'kWhRequested': 65.45, 'mi..."
7,7,5e23b149f9af8b5fe4b973d6,2020-01-02 14:17:32+00:00,2020-01-03 00:44:54+00:00,2020-01-02 20:22:40+00:00,32.441,1_1_191_804_2020-01-02 14:17:32.329535,1,AG-4F34,1-1-191-804,America/Los_Angeles,3737.0,"[{'WhPerMile': 300, 'kWhRequested': 45.0, 'mil..."
8,8,5e23b149f9af8b5fe4b973d7,2020-01-02 14:25:38+00:00,2020-01-02 19:36:05+00:00,2020-01-02 18:38:39+00:00,13.264,1_1_194_826_2020-01-02 14:25:37.578692,1,AG-1F11,1-1-194-826,America/Los_Angeles,419.0,"[{'WhPerMile': 400, 'kWhRequested': 40.0, 'mil..."
9,9,5e23b149f9af8b5fe4b973d8,2020-01-02 14:27:40+00:00,2020-01-02 20:01:48+00:00,2020-01-02 19:17:46+00:00,13.285,1_1_178_823_2020-01-02 14:27:39.582337,1,AG-1F08,1-1-178-823,America/Los_Angeles,651.0,"[{'WhPerMile': 400, 'kWhRequested': 20.0, 'mil..."


# TODO

- check validity and meaning of data (userInput)
- remove irrelevant columns
- split userInput in own columns
- check for duplicates
- check for None-values (userID)
- split userID into binary dummy (isRegistered)
- check for None-values for isRegistered = 1 (remove None-values for registered users)
- check data types of columns
- look at columns with multiple data types (remove wrong entries)
- check ranges of columns & unique values

# Data Cleanup

## Step 1: check validity of columns

- remove unnecessary "unnamed" column
- sessionID does not bring any relevant information for our analysis

In [5]:
# remove irrelevant columns
charging_set.drop('Unnamed: 0', axis=1, inplace=True)
charging_set.drop('sessionID', axis=1, inplace=True) # does not bring relevant information for our analysis
charging_set.head(3)

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,spaceID,stationID,timezone,userID,userInputs
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,"[{'WhPerMile': 250, 'kWhRequested': 25.0, 'mil..."
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,"[{'WhPerMile': 280, 'kWhRequested': 70.0, 'mil..."
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,"[{'WhPerMile': 400, 'kWhRequested': 8.0, 'mile..."


In [6]:
# split userInputs into own columns
def parse_user_inputs(row):
    try:
        # Parse the row as a Python object (e.g., list of dicts)
        parsed = ast.literal_eval(row) if pd.notnull(row) else []
        return parsed[0] if isinstance(parsed, list) and len(parsed) > 0 else {}
    except (ValueError, SyntaxError):
        return {}

# Parse 'userInputs' column in charging_set
charging_set['userInputs'] = charging_set['userInputs'].apply(parse_user_inputs)

# Specify the desired features to extract
desired_features = [
    'WhPerMile', 'kWhRequested', 'milesRequested',
    'minutesAvailable', 'modifiedAt', 'paymentRequired',
    'requestedDeparture'
]

# Extract desired features and create new columns
for feature in desired_features:
    charging_set[feature] = charging_set['userInputs'].apply(
        lambda x: x.get(feature) if isinstance(x, dict) else None
    )
# TODO: has to be commented so a cell further down in this notebook can run (it needs userInputs)
charging_set = charging_set.drop(columns=['userInputs'])
charging_set

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,spaceID,stationID,timezone,userID,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,250.0,25.00,100.0,463.0,"Thu, 02 Jan 2020 13:09:39 GMT",True,"Thu, 02 Jan 2020 20:51:54 GMT"
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,280.0,70.00,250.0,595.0,"Thu, 02 Jan 2020 13:37:11 GMT",True,"Thu, 02 Jan 2020 23:31:50 GMT"
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,400.0,8.00,20.0,60.0,"Thu, 02 Jan 2020 13:57:17 GMT",True,"Thu, 02 Jan 2020 14:56:35 GMT"
3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,400.0,8.00,20.0,65.0,"Thu, 02 Jan 2020 14:00:03 GMT",True,"Thu, 02 Jan 2020 15:04:58 GMT"
4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,400.0,16.00,40.0,504.0,"Thu, 02 Jan 2020 14:00:13 GMT",True,"Thu, 02 Jan 2020 22:24:01 GMT"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66445,5d574ad2f9af8b4c10c03652,2019-07-31 18:08:04+00:00,2019-07-31 23:29:18+00:00,2019-07-31 23:30:18+00:00,28.787,1,AG-3F27,1-1-179-809,America/Los_Angeles,393.0,240.0,31.20,130.0,355.0,"Wed, 31 Jul 2019 18:08:23 GMT",True,"Thu, 01 Aug 2019 00:03:04 GMT"
66446,5d574ad2f9af8b4c10c03653,2019-07-31 18:40:41+00:00,2019-08-01 00:59:42+00:00,2019-07-31 21:44:23+00:00,7.787,1,AG-3F30,1-1-179-810,America/Los_Angeles,220.0,333.0,6.66,20.0,455.0,"Wed, 31 Jul 2019 18:41:02 GMT",True,"Thu, 01 Aug 2019 02:15:41 GMT"
66447,5d574ad2f9af8b4c10c03654,2019-07-31 19:04:40+00:00,2019-07-31 22:44:22+00:00,2019-07-31 22:45:21+00:00,11.274,1,AG-4F51,1-1-191-795,America/Los_Angeles,1974.0,333.0,19.98,60.0,184.0,"Wed, 31 Jul 2019 19:04:57 GMT",True,"Wed, 31 Jul 2019 22:08:40 GMT"
66448,5d574ad2f9af8b4c10c03655,2019-07-31 19:19:47+00:00,2019-08-01 00:34:51+00:00,2019-07-31 21:25:30+00:00,11.589,1,AG-4F43,1-1-191-778,America/Los_Angeles,942.0,275.0,22.00,80.0,60.0,"Wed, 31 Jul 2019 19:20:10 GMT",True,"Wed, 31 Jul 2019 20:19:47 GMT"


In [7]:
# check for duplicates
duplicate_counter = charging_set.duplicated().sum()
duplicates = charging_set[charging_set.duplicated()]
print("sum of duplicates:", duplicate_counter)
charging_set.drop_duplicates()

sum of duplicates: 1413


,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,spaceID,stationID,timezone,userID,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,AG-3F30,1-1-179-810,America/Los_Angeles,194.0,250.0,25.00,100.0,463.0,"Thu, 02 Jan 2020 13:09:39 GMT",True,"Thu, 02 Jan 2020 20:51:54 GMT"
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,AG-1F01,1-1-193-825,America/Los_Angeles,4275.0,280.0,70.00,250.0,595.0,"Thu, 02 Jan 2020 13:37:11 GMT",True,"Thu, 02 Jan 2020 23:31:50 GMT"
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,AG-1F03,1-1-193-829,America/Los_Angeles,344.0,400.0,8.00,20.0,60.0,"Thu, 02 Jan 2020 13:57:17 GMT",True,"Thu, 02 Jan 2020 14:56:35 GMT"
3,5e23b149f9af8b5fe4b973d2,2020-01-02 13:59:58+00:00,2020-01-02 16:38:39+00:00,2020-01-02 15:18:45+00:00,2.355,1,AG-1F04,1-1-193-820,America/Los_Angeles,1117.0,400.0,8.00,20.0,65.0,"Thu, 02 Jan 2020 14:00:03 GMT",True,"Thu, 02 Jan 2020 15:04:58 GMT"
4,5e23b149f9af8b5fe4b973d3,2020-01-02 14:00:01+00:00,2020-01-02 22:08:40+00:00,2020-01-02 18:17:30+00:00,13.375,1,AG-1F06,1-1-193-819,America/Los_Angeles,334.0,400.0,16.00,40.0,504.0,"Thu, 02 Jan 2020 14:00:13 GMT",True,"Thu, 02 Jan 2020 22:24:01 GMT"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65032,5d2fbdd3f9af8b4d0dd0d54a,2019-07-01 21:49:12+00:00,2019-07-02 01:37:18+00:00,2019-07-01 23:14:33+00:00,8.399,1,AG-3F18,1-1-179-798,America/Los_Angeles,1346.0,400.0,32.00,80.0,36.0,"Mon, 01 Jul 2019 21:49:37 GMT",True,"Mon, 01 Jul 2019 22:25:12 GMT"
65033,5d2fbdd3f9af8b4d0dd0d54b,2019-07-01 21:58:45+00:00,2019-07-02 00:39:48+00:00,2019-07-02 00:40:21+00:00,16.864,1,AG-3F20,1-1-179-794,America/Los_Angeles,364.0,400.0,40.00,100.0,234.0,"Mon, 01 Jul 2019 21:59:09 GMT",True,"Tue, 02 Jul 2019 01:52:45 GMT"
65034,5d2fbdd3f9af8b4d0dd0d54c,2019-07-01 22:02:21+00:00,2019-07-02 00:58:50+00:00,2019-07-02 00:59:23+00:00,18.335,1,AG-4F47,1-1-191-807,America/Los_Angeles,2050.0,333.0,29.97,90.0,309.0,"Mon, 01 Jul 2019 22:02:37 GMT",True,"Tue, 02 Jul 2019 03:11:21 GMT"
65035,5d2fbdd3f9af8b4d0dd0d54d,2019-07-01 22:23:44+00:00,2019-07-02 02:03:36+00:00,2019-07-02 02:04:06+00:00,22.815,1,AG-3F31,1-1-179-781,America/Los_Angeles,1626.0,200.0,34.00,170.0,99.0,"Mon, 01 Jul 2019 22:23:43 GMT",True,"Tue, 02 Jul 2019 00:02:44 GMT"


In [8]:
# check for none-values
nan_counts = charging_set.isna().sum()
nan_counts_sorted = nan_counts.sort_values(ascending=False)

for column, count in nan_counts_sorted.items():
    print(f"Feature: {column}, Count of NaN-Values: {count}")

Feature: requestedDeparture, Count of NaN-Values: 17263
Feature: milesRequested, Count of NaN-Values: 17263
Feature: paymentRequired, Count of NaN-Values: 17263
Feature: modifiedAt, Count of NaN-Values: 17263
Feature: minutesAvailable, Count of NaN-Values: 17263
Feature: WhPerMile, Count of NaN-Values: 17263
Feature: kWhRequested, Count of NaN-Values: 17263
Feature: userID, Count of NaN-Values: 17263
Feature: doneChargingTime, Count of NaN-Values: 4088
Feature: id, Count of NaN-Values: 0
Feature: timezone, Count of NaN-Values: 0
Feature: spaceID, Count of NaN-Values: 0
Feature: stationID, Count of NaN-Values: 0
Feature: kWhDelivered, Count of NaN-Values: 0
Feature: disconnectTime, Count of NaN-Values: 0
Feature: connectionTime, Count of NaN-Values: 0
Feature: siteID, Count of NaN-Values: 0


In [9]:
### check data types of each column
# Find the unique types
# TODO: add userInputs columns into here
id_types = charging_set["id"].apply(type).unique()
connectionTime_types = charging_set["connectionTime"].apply(type).unique()
disconnectTime_types = charging_set["disconnectTime"].apply(type).unique()
doneChargingTime_types = charging_set["doneChargingTime"].apply(type).unique()
kWhDelivered_types = charging_set["kWhDelivered"].apply(type).unique()
siteID_types = charging_set["siteID"].apply(type).unique()
spaceID_types = charging_set["spaceID"].apply(type).unique()
stationID_types = charging_set["stationID"].apply(type).unique()
timezone_types = charging_set["timezone"].apply(type).unique()
userID_types = charging_set["userID"].apply(type).unique()

# Print the unique types
print("id types:", id_types)
print("connectionTime types:", connectionTime_types)
print("disconnectTime types:", disconnectTime_types)
print("doneChargingTime types:", doneChargingTime_types)
print("kWhDelivered types:", kWhDelivered_types)
print("siteID types:", siteID_types)
print("spaceID types:", spaceID_types)
print("stationID types:", stationID_types)
print("timezone types:", timezone_types)
print("userID types:", userID_types)

id types: [<class 'str'>]
connectionTime types: [<class 'str'>]
disconnectTime types: [<class 'str'>]
doneChargingTime types: [<class 'str'> <class 'float'>]
kWhDelivered types: [<class 'float'>]
siteID types: [<class 'int'>]
spaceID types: [<class 'str'>]
stationID types: [<class 'str'>]
timezone types: [<class 'str'>]
userID types: [<class 'float'>]


In [10]:
##### check ranges of column values
# TODO: replace None values with None-placeholders, then this cell also runs smoothly
# print("id data range:", np.sort(charging_set["id"].unique()))
# print("connectionTime data range:", np.sort(charging_set["connectionTime"].unique()))
# print("disconnectTime data range:", np.sort(charging_set["disconnectTime"].unique()))
# print("doneChargingTime data range:", np.sort(charging_set["doneChargingTime"].unique()))
# print("kWhDelivered data range:", np.sort(charging_set["kWhDelivered"].unique()))
# print("sessionID data range:", np.sort(charging_set["sessionID"].unique()))
# print("siteID data range:", np.sort(charging_set["siteID"].unique()))
# print("spaceID data range:", np.sort(charging_set["spaceID"].unique()))
# print("stationID data range:", np.sort(charging_set["stationID"].unique()))
# print("timezone data range:", np.sort(charging_set["timezone"].unique()))
# print("userID data range:", np.sort(charging_set["userID"].unique()))
# print("userInputs data range:", np.sort(charging_set["userInputs"].unique()))
# print("connectionTime_date data range:", np.sort(charging_set["connectionTime_date"].unique()))
# print("connectionTime_time data range:", np.sort(charging_set["connectionTime_time"].unique()))
# print("disconnectTime_date data range:", np.sort(charging_set["disconnectTime_date"].unique()))
# print("disconnectTime_time data range:", np.sort(charging_set["disconnectTime_time"].unique()))
# print("doneChargingTime_date data range:", np.sort(charging_set["doneChargingTime_date"].unique()))
# print("doneChargingTime_time data range:", np.sort(charging_set["doneChargingTime_time"].unique()))

Since the "timezone" column only has the value "America/Los Angeles", we can remove this column.

In [11]:
charging_set.drop('timezone', axis=1, inplace=True)
charging_set.head(3)

,id,connectionTime,disconnectTime,doneChargingTime,kWhDelivered,siteID,spaceID,stationID,userID,WhPerMile,kWhRequested,milesRequested,minutesAvailable,modifiedAt,paymentRequired,requestedDeparture
0,5e23b149f9af8b5fe4b973cf,2020-01-02 13:08:54+00:00,2020-01-02 19:11:15+00:00,2020-01-02 17:31:35+00:00,25.016,1,AG-3F30,1-1-179-810,194.0,250.0,25.0,100.0,463.0,"Thu, 02 Jan 2020 13:09:39 GMT",True,"Thu, 02 Jan 2020 20:51:54 GMT"
1,5e23b149f9af8b5fe4b973d0,2020-01-02 13:36:50+00:00,2020-01-02 22:38:21+00:00,2020-01-02 20:18:05+00:00,33.097,1,AG-1F01,1-1-193-825,4275.0,280.0,70.0,250.0,595.0,"Thu, 02 Jan 2020 13:37:11 GMT",True,"Thu, 02 Jan 2020 23:31:50 GMT"
2,5e23b149f9af8b5fe4b973d1,2020-01-02 13:56:35+00:00,2020-01-03 00:39:22+00:00,2020-01-02 16:35:06+00:00,6.521,1,AG-1F03,1-1-193-829,344.0,400.0,8.0,20.0,60.0,"Thu, 02 Jan 2020 13:57:17 GMT",True,"Thu, 02 Jan 2020 14:56:35 GMT"


Next step: analyze "connectionTime", "disconnectTime" and "doneChargingTime", since these columns should hold time-datatypes and not strings nor floats (except for NaN-values since these have the type float).

# Time Data Preparation/Feature Engineering (draft)

For better data understanding and to improve data accessibility and interpretability, it is important to convert the current time data format into a human-readable time datatype. Currently the time related columns ("connectionTime", "disconnectionTime", "doneChargingTime") are formatted in the UTC format. To prepare the data for further interpretation we need to gather the relevant information. In this case the important information is the charging duration.

To get the duration we divide each of the time related columns into separate date and datetime columns. With these columns, the difference between connecting the EV with the station ("connectionTime") and disconnecting the EV with the station ("disconnectTime") can be calculated and stored in a separate column ("chargingDuration").


In [12]:
### split connectionTime in separate day and time columns
# Convert the column to datetime format
charging_set['connectionTime'] = pd.to_datetime(charging_set['connectionTime'])

# Create a new column for the date
charging_set['connectionTime_date'] = charging_set['connectionTime'].dt.date

# Create a new column for the time of the day
charging_set['connectionTime_time'] = charging_set['connectionTime'].dt.time


### split disconnectTime in separate day and time columns
# Convert the column to datetime format
charging_set['disconnectTime'] = pd.to_datetime(charging_set['disconnectTime'])

# Create a new column for the date
charging_set['disconnectTime_date'] = charging_set['disconnectTime'].dt.date

# Create a new column for the time of the day
charging_set['disconnectTime_time'] = charging_set['disconnectTime'].dt.time


### split doneChargingTime in separate day and time columns
# Convert the column to datetime format
charging_set['doneChargingTime'] = pd.to_datetime(charging_set['doneChargingTime'])

# Create a new column for the date
charging_set['doneChargingTime_date'] = charging_set['doneChargingTime'].dt.date

# Create a new column for the time of the day
charging_set['doneChargingTime_time'] = charging_set['doneChargingTime'].dt.time


### split disconnectTime in separate day and time columns
# Convert the column to datetime format
charging_set['modifiedAt'] = pd.to_datetime(charging_set['modifiedAt'])

# Create a new column for the date
charging_set['modifiedAt_date'] = charging_set['modifiedAt'].dt.date

# Create a new column for the time of the day
charging_set['modifiedAt_time'] = charging_set['modifiedAt'].dt.time

### split disconnectTime in separate day and time columns
# Convert the column to datetime format
charging_set['requestedDeparture'] = pd.to_datetime(charging_set['requestedDeparture'])

# Create a new column for the date
charging_set['requestedDeparture_date'] = charging_set['requestedDeparture'].dt.date

# Create a new column for the time of the day
charging_set['requestedDeparture_time'] = charging_set['requestedDeparture'].dt.time

In [13]:
### convert date columns into separate year, month and day columns

# Das muss rein, weil sonst kommt ein Fehler
charging_set['connectionTime_date'] = pd.to_datetime(charging_set['connectionTime_date'])
charging_set['disconnectTime_date'] = pd.to_datetime(charging_set['disconnectTime_date'])
charging_set['doneChargingTime_date'] = pd.to_datetime(charging_set['doneChargingTime_date'])
charging_set['modifiedAt_date'] = pd.to_datetime(charging_set['modifiedAt_date'])
charging_set['requestedDeparture_date'] = pd.to_datetime(charging_set['requestedDeparture_date'])

charging_set['connectionTime_year'] = charging_set['connectionTime_date'].dt.year
charging_set['connectionTime_month'] = charging_set['connectionTime_date'].dt.month
charging_set['connectionTime_day'] = charging_set['connectionTime_date'].dt.day

charging_set['disconnectTime_year'] = charging_set['disconnectTime_date'].dt.year
charging_set['disconnectTime_month'] = charging_set['disconnectTime_date'].dt.month
charging_set['disconnectTime_day'] = charging_set['disconnectTime_date'].dt.day

charging_set['doneChargingTime_year'] = charging_set['doneChargingTime_date'].dt.year
charging_set['doneChargingTime_month'] = charging_set['doneChargingTime_date'].dt.month
charging_set['doneChargingTime_day'] = charging_set['doneChargingTime_date'].dt.day

charging_set['modifiedAt_year'] = charging_set['modifiedAt_date'].dt.year
charging_set['modifiedAt_month'] = charging_set['modifiedAt_date'].dt.month
charging_set['modifiedAt_day'] = charging_set['modifiedAt_date'].dt.day
charging_set['modifiedAt_time'] = charging_set['modifiedAt_date'].dt.time
# charging_set['modifiedAt_weekday'] = charging_set['modifiedAt_date'].dt.weekday

charging_set['requestedDeparture_year'] = charging_set['requestedDeparture_date'].dt.year
charging_set['requestedDeparture_month'] = charging_set['requestedDeparture_date'].dt.month
charging_set['requestedDeparture_day'] = charging_set['requestedDeparture_date'].dt.day
# charging_set['requestedDeparture_weekday'] = charging_set['requestedDeparture_date'].dt.weekday

In [14]:
def calculate_hours():
    # Berechne die Ladezeit als Differenz zwischen disconnectTime und connectionTime
    charging_set["chargingDurationTemp"] = charging_set["disconnectTime"] - charging_set["connectionTime"]
    
    # Konvertiere die Differenz in Stunden
    charging_set['chargingDuration'] = charging_set["chargingDurationTemp"].apply(
        lambda x: x.total_seconds() / 3600
    )
    
    # Entferne die temporäre Spalte
    charging_set.drop(columns=["chargingDurationTemp"], inplace=True)
    
    return charging_set

# Aufruf der Funktion
charging_set = calculate_hours()

In [15]:
# clean "connectionTime", "disconnectTime", "doneChargingTime" from dataframe
charging_set = charging_set.drop(columns=["connectionTime", "disconnectTime", "doneChargingTime"])

Idee: 
- entferne temporär erstmal alle nan Werte um zum checken, ob die Funktion überhaupt funktioniert
- erstelle dafür eine temp datei
- danach sollte man sich mit dem auffüllen / entfernen von NaNs beschäftigen

In [16]:
nan_user_id_indices = charging_set[charging_set['userID'].isna()].index[:10]

# Display the indices
print(nan_user_id_indices)

Index([11, 41, 180, 193, 265, 276, 407, 457, 474, 500], dtype='int64')


In [17]:
charging_set.loc[11]

id                          5e23b149f9af8b5fe4b973da
kWhDelivered                                   0.812
siteID                                             1
spaceID                                      AG-1F14
stationID                                1-1-194-818
userID                                           NaN
WhPerMile                                        NaN
kWhRequested                                     NaN
milesRequested                                   NaN
minutesAvailable                                 NaN
modifiedAt                                       NaT
paymentRequired                                 None
requestedDeparture                               NaT
connectionTime_date              2020-01-02 00:00:00
connectionTime_time                         15:04:38
disconnectTime_date              2020-01-02 00:00:00
disconnectTime_time                         23:33:26
doneChargingTime_date            2020-01-02 00:00:00
doneChargingTime_time                       22

In [18]:
charging_set['UserID'] = charging_set['userID'].apply(lambda x: 1 if pd.notna(x) else 0)

In [19]:
nan_chargingTime = charging_set[charging_set['doneChargingTime_time'].isna()].index[:10]

# Display the indices
print(nan_chargingTime)


Index([3756, 3757, 3758, 3759, 3760, 3761, 3762, 3763, 3764, 3765], dtype='int64')


In [20]:
charging_set.head(30)

,id,kWhDelivered,siteID,spaceID,stationID,userID,WhPerMile,kWhRequested,milesRequested,minutesAvailable,...,doneChargingTime_month,doneChargingTime_day,modifiedAt_year,modifiedAt_month,modifiedAt_day,requestedDeparture_year,requestedDeparture_month,requestedDeparture_day,chargingDuration,UserID
0,5e23b149f9af8b5fe4b973cf,25.016,1,AG-3F30,1-1-179-810,194.0,250.0,25.00,100.0,463.0,...,1.0,2.0,2020.0,1.0,2.0,2020.0,1.0,2.0,6.039167,1
1,5e23b149f9af8b5fe4b973d0,33.097,1,AG-1F01,1-1-193-825,4275.0,280.0,70.00,250.0,595.0,...,1.0,2.0,2020.0,1.0,2.0,2020.0,1.0,2.0,9.025278,1
2,5e23b149f9af8b5fe4b973d1,6.521,1,AG-1F03,1-1-193-829,344.0,400.0,8.00,20.0,60.0,...,1.0,2.0,2020.0,1.0,2.0,2020.0,1.0,2.0,10.713056,1
3,5e23b149f9af8b5fe4b973d2,2.355,1,AG-1F04,1-1-193-820,1117.0,400.0,8.00,20.0,65.0,...,1.0,2.0,2020.0,1.0,2.0,2020.0,1.0,2.0,2.644722,1
4,5e23b149f9af8b5fe4b973d3,13.375,1,AG-1F06,1-1-193-819,334.0,400.0,16.00,40.0,504.0,...,1.0,2.0,2020.0,1.0,2.0,2020.0,1.0,2.0,8.144167,1
5,5e23b149f9af8b5fe4b973d4,13.370,1,AG-1F13,1-1-194-821,3519.0,600.0,24.00,40.0,624.0,...,1.0,2.0,2020.0,1.0,2.0,2020.0,1.0,3.0,9.007778,1
6,5e23b149f9af8b5fe4b973d5,43.477,1,AG-1F09,1-1-178-817,933.0,385.0,65.45,170.0,647.0,...,1.0,3.0,2020.0,1.0,2.0,2020.0,1.0,3.0,11.812222,1
7,5e23b149f9af8b5fe4b973d6,32.441,1,AG-4F34,1-1-191-804,3737.0,300.0,45.00,150.0,126.0,...,1.0,2.0,2020.0,1.0,2.0,2020.0,1.0,2.0,10.456111,1
8,5e23b149f9af8b5fe4b973d7,13.264,1,AG-1F11,1-1-194-826,419.0,400.0,40.00,100.0,491.0,...,1.0,2.0,2020.0,1.0,2.0,2020.0,1.0,2.0,5.174167,1
9,5e23b149f9af8b5fe4b973d8,13.285,1,AG-1F08,1-1-178-823,651.0,400.0,20.00,50.0,226.0,...,1.0,2.0,2020.0,1.0,2.0,2020.0,1.0,2.0,5.568889,1


In [21]:
charging_set.columns

Index(['id', 'kWhDelivered', 'siteID', 'spaceID', 'stationID', 'userID',
       'WhPerMile', 'kWhRequested', 'milesRequested', 'minutesAvailable',
       'modifiedAt', 'paymentRequired', 'requestedDeparture',
       'connectionTime_date', 'connectionTime_time', 'disconnectTime_date',
       'disconnectTime_time', 'doneChargingTime_date', 'doneChargingTime_time',
       'modifiedAt_date', 'modifiedAt_time', 'requestedDeparture_date',
       'requestedDeparture_time', 'connectionTime_year',
       'connectionTime_month', 'connectionTime_day', 'disconnectTime_year',
       'disconnectTime_month', 'disconnectTime_day', 'doneChargingTime_year',
       'doneChargingTime_month', 'doneChargingTime_day', 'modifiedAt_year',
       'modifiedAt_month', 'modifiedAt_day', 'requestedDeparture_year',
       'requestedDeparture_month', 'requestedDeparture_day',
       'chargingDuration', 'UserID'],
      dtype='object')

In [22]:
random_entry = charging_set.sample(n=1).iloc[0]

# Zellenwerte einzeln ausgeben
for column_name, value in random_entry.items():
    print(f"{column_name}: {value}")

id: 5bc93eacf9af8b0dc677dcb4
kWhDelivered: 13.196
siteID: 2
spaceID: CA-323
stationID: 2-39-78-366
userID: nan
WhPerMile: nan
kWhRequested: nan
milesRequested: nan
minutesAvailable: nan
modifiedAt: NaT
paymentRequired: None
requestedDeparture: NaT
connectionTime_date: 2018-09-20 00:00:00
connectionTime_time: 00:41:03
disconnectTime_date: 2018-09-20 00:00:00
disconnectTime_time: 13:04:07
doneChargingTime_date: 2018-09-20 00:00:00
doneChargingTime_time: 06:06:37
modifiedAt_date: NaT
modifiedAt_time: NaT
requestedDeparture_date: NaT
requestedDeparture_time: NaT
connectionTime_year: 2018
connectionTime_month: 9
connectionTime_day: 20
disconnectTime_year: 2018
disconnectTime_month: 9
disconnectTime_day: 20
doneChargingTime_year: 2018.0
doneChargingTime_month: 9.0
doneChargingTime_day: 20.0
modifiedAt_year: nan
modifiedAt_month: nan
modifiedAt_day: nan
requestedDeparture_year: nan
requestedDeparture_month: nan
requestedDeparture_day: nan
chargingDuration: 12.384444444444444
UserID: 0


In [23]:
charging_set = charging_set.drop(columns=['modifiedAt', 'requestedDeparture', 'connectionTime_date', 'disconnectTime_date', 'doneChargingTime_date', 'modifiedAt_date', 'requestedDeparture_date'])

In [24]:
random_entry = charging_set.sample(n=1).iloc[0]

# Zellenwerte einzeln ausgeben
for column_name, value in random_entry.items():
    print(f"{column_name}: {value}")

id: 5d4b6d54f9af8b256a1e0d1b
kWhDelivered: 9.957
siteID: 1
spaceID: AG-3F15
stationID: 1-1-179-813
userID: 1461.0
WhPerMile: 600.0
kWhRequested: 48.0
milesRequested: 80.0
minutesAvailable: 419.0
paymentRequired: True
connectionTime_time: 13:49:28
disconnectTime_time: 23:06:49
doneChargingTime_time: 16:20:22
modifiedAt_time: 00:00:00
requestedDeparture_time: 20:48:28
connectionTime_year: 2019
connectionTime_month: 7
connectionTime_day: 22
disconnectTime_year: 2019
disconnectTime_month: 7
disconnectTime_day: 22
doneChargingTime_year: 2019.0
doneChargingTime_month: 7.0
doneChargingTime_day: 22.0
modifiedAt_year: 2019.0
modifiedAt_month: 7.0
modifiedAt_day: 22.0
requestedDeparture_year: 2019.0
requestedDeparture_month: 7.0
requestedDeparture_day: 22.0
chargingDuration: 9.289166666666667
UserID: 1


In [25]:
# charging_set.to_csv('data/cleaned_charging_set.csv', index=False)